让车动起来的代码

env.step()
->  road.act() # road/road.py
    ->  vehicle.act() # vehicle/controller.py  cls MDPVehilce
    ->  vehicle.act() # vehicle/behavior.py  cls IDMVehicle

获取环境车的状态空间   车的序号如下
————————————————————————————————————————————>
44444                              333333
-———————————————————————————————————————————>
2222           000000              11111
-———————————————————————————————————————————>
66666                              55555
-———————————————————————————————————————————>
 

需要获取环境车状态空间
新增函数 
IMDVehicle.d_get_state_vehicle() -> 返回7辆车
IMDVehicle.d_get_state() 返回状态



In [ ]:
def d_get_state_vehicle(self):
    """Get the state of the vehicle."""
    front_vehicle, rear_vehicle = self.road.neighbour_vehicles(
        self, self.lane_index
    )
    """Left"""


    Lanes =  self.road.network.d_side_lanes(self.lane_index)
    Leftline = Lanes[0]
    Rightline = Lanes[1]
    if Leftline is not None:
        front_left_vehicle, rear_left_vehicle = self.road.neighbour_vehicles(
            self, Leftline
        )
    else:
        front_left_vehicle, rear_left_vehicle = None, None
    if Rightline is not None:
        front_right_vehicle, rear_right_vehicle = self.road.neighbour_vehicles(
            self, Rightline
        )
    else:
        front_right_vehicle, rear_right_vehicle = None, None
    state_v = [self,
                front_vehicle,
                rear_vehicle,
                front_left_vehicle,
                rear_left_vehicle,
                front_right_vehicle,
                rear_right_vehicle]
    return state_v

新增函数 
def d_side_lanes(self, lane_index: LaneIndex) -> List[LaneIndex]:
与原文区别是，当不存在的时候返回None

In [ ]:
def d_side_lanes(self, lane_index: LaneIndex) -> List[LaneIndex]:
    """
    :param lane_index: the index of a lane.
    :return: indexes of lanes next to a an input lane, to its right or left.
    """
    _from, _to, _id = lane_index
    lanes = []
    if _id < len(self.graph[_from][_to]) - 1:
        lanes.append((_from, _to, _id + 1))
    else:
        lanes.append(None)
    if _id > 0:
        lanes.append((_from, _to, _id - 1))
    else:
        lanes.append(None)
    return lanes